In [ ]:
#Pradeep Wijaywickrama
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
 !pip install --user datasets tqdm pandas

In [ ]:
!pip install --user sentencepiece

In [ ]:
!pip install --user accelerate>=0.20.1
!pip install --user torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install --user transformers[torch]

In [ ]:
!pip install --user wandb

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
#Import Packages
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import sentencepiece as spm

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
import random
import numpy as np
import torch
import datasets

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('D:/Personal/Edu/Study/FYP/NLP/data_set/c4_200m_550k.csv')
df.shape

In [ ]:
df.head()

In [ ]:
# Importing classes and modules from the Transformers library
from transformers import (
    T5ForConditionalGeneration,  # T5 model for text generation
    T5Tokenizer,  # Tokenizer for T5 model
    Seq2SeqTrainingArguments,  # Training configuration for seq2seq tasks
    Seq2SeqTrainer,  # Trainer for seq2seq models
    DataCollatorForSeq2Seq  # Data collator for seq2seq tasks
)

# Importing necessary classes from PyTorch
from torch.utils.data import Dataset, DataLoader


In [ ]:
# Define the model name as 't5-base'
model_name = 't5-base'

# Create a tokenizer for the T5 model using the pre-trained 't5-base' model
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Instantiate a T5 model for conditional text generation with pre-trained weights ('t5-base')
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
def calc_token_len(example):
    """
    Calculate the number of tokens in a tokenized input example.

    Parameters:
    - example (str): Input example to be tokenized.

    Returns:
    - int: Number of tokens in the tokenized input.
    """
    # Tokenize the input example using the specified tokenizer
    tokenized_output = tokenizer(example)
    
    # Retrieve the token IDs from the tokenized output
    token_ids = tokenized_output.input_ids
    
    # Calculate the length of the list of token IDs, representing the number of tokens
    num_tokens = len(token_ids)
    
    # Return the number of tokens
    return num_tokens


In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the DataFrame 'df' into training and testing sets
# - test_size=0.10: Specifies that 10% of the data will be used for testing
# - shuffle=True: Randomly shuffles the data before splitting
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape


In [ ]:
# Apply the 'calc_token_len' function to each 'input' text and create a new column 'input_token_len' in 'test_df'
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [ ]:
test_df.head()

In [ ]:
# Display descriptive statistics for the 'input_token_len' column in 'test_df'
test_df['input_token_len'].describe()

In [ ]:
from datasets import Dataset
# Create datasets from Pandas DataFrames for training and testing
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
test_dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer, print_text=False):
        """
        Initializes the GrammarDataset.

        Parameters:
        - dataset (pandas.DataFrame): The input dataset containing 'input' and 'output' columns.
        - tokenizer (transformers.PreTrainedTokenizer): The tokenizer for tokenizing input and target texts.
        - print_text (bool): Flag to print tokenized text lengths during dataset creation.
        """
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        """
        Returns the total number of examples in the dataset.
        """
        return len(self.dataset)

    def tokenize_data(self, example):
        """
        Tokenizes input and target texts for a given example.

        Parameters:
        - example (dict): A dictionary with 'input' and 'output' keys.

        Returns:
        - dict: A dictionary containing tokenized input_ids, attention_mask, and labels.
        """
        input_, target_ = example['input'], example['output']

        # Tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                     max_length=self.max_len,
                                     return_attention_mask=True)

        # Tokenize targets
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                      max_length=self.max_len,
                                      return_attention_mask=True)

        # Prepare dictionary with tokenized inputs, attention_mask, and labels
        inputs = {
            "input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs

    def __getitem__(self, index):
        """
        Retrieves a tokenized example for a given index.

        Parameters:
        - index (int): Index of the example to retrieve.

        Returns:
        - dict: A dictionary containing tokenized input_ids, attention_mask, and labels.
        """
        # Tokenize data for the given index
        inputs = self.tokenize_data(self.dataset[index])

        # Print tokenized text lengths if specified
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs


In [ ]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

In [ ]:
!pip install rouge_score

In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")
rouge_metric

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [ ]:
# defining training related arguments
"""
output_dir:

Specifies the directory where the model and training outputs will be saved.
evaluation_strategy="steps":

Specifies that evaluation should be performed at regular steps during training.
per_device_train_batch_size and per_device_eval_batch_size:

Sets the batch size per GPU for both training and evaluation.
learning_rate=2e-5:

Sets the initial learning rate for the optimizer.
num_train_epochs=1:

Specifies the number of training epochs.
weight_decay=0.01:

Sets the weight decay for regularization.
save_total_limit=2:

Limits the total number of checkpoints saved during training.
predict_with_generate=True:

Enables generation during evaluation.
fp16=False:

Disables mixed-precision training (float16).
gradient_accumulation_steps=6:

Accumulates gradients over multiple steps to effectively simulate a larger batch size.
eval_steps=500:

Specifies the evaluation steps interval during training.
save_steps=500:

Specifies the saving checkpoints interval during training.
load_best_model_at_end=True:

Loads the best model at the end of training.
logging_dir="/logs":

Specifies the directory where logs will be stored.
report_to="wandb":

Specifies that training metrics will be reported to Weights & Biases for logging and visualization.

"""
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="D:/Personal/Edu/Study/FYP/NLP/data_set/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = False,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")


In [ ]:

import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# defining trainer using 
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [ ]:
trainer.train()
#  KEY --> 02078263b081615fe0c19877d205d3d75b693792

In [ ]:
trainer.save_model('t5_gec_model')

In [ ]:
import shutil

shutil.make_archive('t5_gec_model', 'zip', 't5_gec_model')

#!zip -r 't5_gec_model.zip' 't5_gec_model'

In [ ]:
#!mv t5_gec_model.zip D:/Personal/Edu/Study/FYP/NLP/data_set/c4_200m
!move t5_gec_model.zip D:\Personal\Edu\Study\FYP\NLP\data_set\c4_200m


In [ ]:
from typing import List
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
def correct_grammar(input_text, num_return_sequences) -> str:
    batch = tokenizer([input_text], truncation=True, padding='max_length', max_length=64, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=64, num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    # Join the list of strings into a single string
    result_string = ' '.join(tgt_text)
    
    return result_string


In [ ]:
text =  'I are Super Man to this universe. I going to moon. we Am goind to develop NLP system. She do not like the way he sing.'
print(correct_grammar(text, num_return_sequences = 1))